## Naive Bayes
- 베이즈 정리 + featuer 간 독립 가정

- 출처 : 딥러닝을 이용한 자연어 처리 입문

데이터 이해

In [ ]:
from sklearn.datasets import fetch_20newsgroups
# Sklearn 의 Twenty Newsgroups 이라는 18,846개 뉴스 그룹 데이터 사용
# 해당 데이터는 train 11,314개, test 7,532개로 분류되어 있음

newsdata=fetch_20newsgroups(subset='train')
# Data Download : subset='all', 'train', 'test'

print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [ ]:
print (len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names), len(newsdata.target))
## 모르는 data 쓸 때, 데이터 개수 확인하기 (모델 학습 중간중간 확인해주기 추천)

11314 11314 20 11314


In [ ]:
print(newsdata.target_names)
# 20개 카테고리 이름 출력 (모르는 data 살펴보기)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [ ]:
print(newsdata.target[0])
# 첫번째 샘플이 7번 카테고리에 속하는 데이터임을 확인 (모르는 data 살펴보기)

7


In [ ]:
print(newsdata.target_names[7])
# 7번 카테고리 이름은 rec.autos (모르는 data 살펴보기)

rec.autos


In [ ]:
print(newsdata.data[0])
# 첫 번째 샘플 데이터 출력하여 내용 확인 - 스포츠카에 관한 설명, 자연어 데이터

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







나이브 베이즈 분류

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [ ]:
dtmvector = CountVectorizer()
# 텍스트 데이터를 BoW로 만드는 CountVectorizer 라이브러리 사용
# 단어 순서 고려X, 문서 내에 단어의 등장 횟수를 고려하여 BoW vector 생성
# 각 단어에 index 지정 후, 각 단어의 등장 횟수를 index에 맞게 배치
# ex) 나는 학교에 가고싶은데 너는 학교에 갈거야? // vocab=['나':0, '는':1, '학교':2, ] (단어리스트) // BoW=[1, 2, 2, ] (단어 등장 횟수)

X_train_dtm = dtmvector.fit_transform(newsdata.data)
# fit_transform() 메서드로 fit과 transform 한 번에 처리

print(X_train_dtm.shape)
# train dataset 개수 11314, 등장한 단어수 130107

(11314, 130107)


In [ ]:
tfidfvector = TfidfVectorizer()
# TF-IDF (Term Frequency Inverse Document Frequency)는 단어의 빈도와 역 문서 빈도의 값을 사용하여 단어들마다 중요한 "가중치"를 주는 방식
# TF : 특성 문서 d에서 특정 단어 t의 등장 횟수 (단어 빈도값)
# DF : 특정 단어 t가 등장한 문서의 수 -> IDF : DF의 반비례 값 // DF 크면 IDF 작음 -> 중요 가중치 낮음 (ex: a, the)
X_train_tfidf = tfidfvector.fit_transform(newsdata.data)
print(X_train_tfidf.shape)

(11314, 130107)


In [ ]:
mod = MultinomialNB(alpha=1.0)
# Multinomial NB는 데이터 특징이 출현 횟수로 표현 (20개 newsdata 분류 -> 범주형 변수)

# 설명 변수가 연속형 변수일 때, Gaussian NB
# 설명 변수가 범주형 변수일 때, Multinomial NB
# 범주형인데, 이진형인 경우 Bernoulli NB

# alpha=1.0 : 모델 복잡도 조절 파라미터
# alpha 가 주어지면, 모든 특성에 양의 값을 가진 가상의 data를 alpha개만큼 추가 (smoothing -> prob.=! 0 되도록 분모에 상수항 더해줌)
mod.fit(X_train_tfidf, newsdata.target)

MultinomialNB()

In [ ]:
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True)
X_test_dtm = dtmvector.transform(newsdata_test.data)
tfidfv_test = tfidfvector.transform(newsdata_test.data)

predicted = mod.predict(tfidfv_test)
print("정확도:", accuracy_score(newsdata_test.target, predicted))
# newssata_test.target = y 와 predicted = y^ (y햇) 비교해서 정확도 파악

## 같은 data라도 어떤 모델을 사용하느냐에 따라서 정확도 다르므로, 여러 모델 사용해서 비교함으로써 최적의 알고리즘 찾기

정확도: 0.7738980350504514
